# JSON Data Type

This tutorial covers the `json` data type in DataJoint, which allows storing semi-structured data within tables. You'll learn:

- When to use the JSON type
- Defining tables with JSON attributes
- Inserting JSON data
- Fetching and working with JSON data

## When to Use JSON

The JSON type is useful when:
- Data structure varies between entries
- You need to store nested or hierarchical data
- The schema is still evolving

**Note:** For querying and filtering, structured columns are more efficient. Use JSON for flexibility, not as a replacement for proper schema design.

In [1]:
import datajoint as dj

# Clean up any existing schema from previous runs
schema = dj.Schema('tutorial_json', create_tables=False)
schema.drop()

# Create fresh schema
schema = dj.Schema('tutorial_json')

[2026-01-22 22:53:39,502][INFO]: DataJoint 2.1.0a5 connected to postgres@localhost:5432


## Table Definition

For this example, imagine we're organizing an RC car race. Each team has a car with varying specifications—some teams track tire pressure, others track weight, and the data structure isn't standardized yet.

This is a good use case for JSON: the `Team` table has a structured primary key, but the `car` details are stored as flexible JSON.

In [2]:
@schema
class Team(dj.Manual):
    definition = """
    # A team in the RC car race
    name: varchar(40)  # team name
    ---
    car=null: json     # car specifications (flexible structure)
    """

## Inserting JSON Data

Insert JSON data using Python dictionaries. The structure can vary between entries.

In [3]:
# Engineering team has detailed specs
Team.insert1({
    "name": "engineering",
    "car": {
        "name": "Rever",
        "length": 20.5,
        "weight": 1.2,
        "tire_pressure": [32, 31, 33, 34],
        "inspected": True
    }
})

In [4]:
# Business team has different specs (no weight, different fields)
Team.insert([
    {
        "name": "business",
        "car": {
            "name": "Chaching",
            "length": 18.0,
            "color": "gold",
            "sponsor": "Acme Corp"
        }
    },
    {
        "name": "marketing",
        "car": None  # Not yet specified
    }
])

## Viewing the Table

In the table preview, JSON columns display as `json` (similar to blob columns).

In [5]:
Team()

name team name,car car specifications (flexible structure)
business,json
engineering,json
marketing,json


## Fetching JSON Data

When you fetch data, JSON columns are automatically deserialized to Python dictionaries.

In [6]:
# Fetch all teams as dictionaries
teams = Team.to_dicts()
for team in teams:
    print(f"{team['name']}: {team['car']}")

engineering: {'name': 'Rever', 'length': 20.5, 'weight': 1.2, 'inspected': True, 'tire_pressure': [32, 31, 33, 34]}
business: {'name': 'Chaching', 'color': 'gold', 'length': 18.0, 'sponsor': 'Acme Corp'}
marketing: None


In [7]:
# Fetch a specific team
eng = (Team & {"name": "engineering"}).fetch1()
print(f"Team: {eng['name']}")
print(f"Car name: {eng['car']['name']}")
print(f"Tire pressures: {eng['car']['tire_pressure']}")

Team: engineering
Car name: Rever
Tire pressures: [32, 31, 33, 34]


In [8]:
# Work with the JSON data in Python
for team in Team.to_dicts():
    car = team['car']
    if car and 'length' in car:
        print(f"{team['name']}'s car is {car['length']} inches long")

engineering's car is 20.5 inches long
business's car is 18.0 inches long


## Updating JSON Data

To update JSON data, delete the old entry and insert the new one (or use `insert` with `replace=True`).

In [9]:
# Marketing finally registered their car
(Team & {"name": "marketing"}).delete_quick()
Team.insert1({
    "name": "marketing",
    "car": {
        "name": "Buzz",
        "length": 22.0,
        "features": ["LED lights", "custom decals"]
    }
})

# Verify the update
(Team & {"name": "marketing"}).fetch1()

{'name': 'marketing',
 'car': {'name': 'Buzz',
  'length': 22.0,
  'features': ['LED lights', 'custom decals']}}

## Summary

| Operation | Method |
|-----------|--------|
| Define JSON column | `attr: json` in definition |
| Insert | Pass Python dict as value |
| Fetch | Returns Python dict automatically |
| Nullable JSON | `attr=null: json` |

### Best Practices

1. **Use JSON sparingly** — Structured columns are better for querying
2. **Document expected structure** — Even flexible data benefits from documentation
3. **Validate in application code** — DataJoint stores any valid JSON
4. **Consider evolution** — JSON is good for data whose structure may change

In [10]:
# Cleanup
schema.drop(prompt=False)